# Physical Reasoning
Physical reasoning refers to ability of humans to reason about physical interactions between objects qualitatively. Building artificial agents which share a similar intuition allows them to adapt to new environment sample efficiently. It also allows them to operate in uncertain environments. Facebook introduced a benchmark called PHYRE to explore this ability, https://phyre.ai/. A similar but more challenging benchmark called The Virtual Tools is introduced by MIT, https://sites.google.com/view/virtualtoolsgame/home.

<br>

### PHYRE
PHYRE benchmark consists of a physics simulator which models gravity and perfect collisions. The central theme of all the tasks is to place one or two balls in the environment such that by the end of the simulation green object touches blue object, which is the goal state. 

(https://player.phyre.ai/#/task/00000:000)

(https://player.phyre.ai/#/task/00006:004)

![Tier B tasks](./media/tasks.png)

<br>

### Details
1. Each tier consists of 25 templates and each template consists of 100 tasks. So across two tiers we have 5000 tasks to learn from.
2. An initial scene of 256 x 256 is provided to the agent for providing context. Each object in the scene is coded by a number which is as follows
    * 0 - Background
    * 1 - Object by agent
    * 2 - Dynamic goal object
    * 3 - Dynamic goal subject
    * 4 - Static goal subject
    * 5 - Dynamic confounding body
    * 6 - Static confounding body

3. An action is performed by sending a tuple (x, y, r) to the simulator. (x, y) represents the position while r represents the radius of ball placed. All these values are between 0 and  1. If an action is outside the scene frame or on any object it results in an invalid action.
4. Simulator returns back with the following reward
    * 1  - Solved the task
    * 0  - Invalid action
    * -1  - Did not solve the task
5. After the simulation other than reward you can request intermediate frames of simulation also.
6. Efficiency of the agent is measured by number of actions that are attempted to solve the task during testing phase. Fewer attempts correspond to greater efficiency. A new metric AUCCESS is coined for emphasis on fewer attempts. AUCCESS is defined as follows

$ AUCCESS =  \frac {\sum_k w_k . s_k} {\sum_k w_k}$

$ w_k = log(k + 1) - log(k)  \quad k \in \{1,...100\}$

$ s_k $ is success percentage at k attempts

In Code

https://github.com/facebookresearch/phyre/blob/2c5247944cfbf9730159da9181d1d6a937362dce/src/python/phyre/metrics.py#L329

### Experiments in the paper
The following agents were developed in paper to establish baselines

1. RAND - Random agent which uniformly samples valid actions
2. MEM - Remembers which random action worked during training and uses them during inference
3. MEM-O - Continues to train during testing phase also
4. DQN - Deep Q-Network which trained by minimizing the cross-entropy between the soft prediction and the observed reward. There are so many finer details here.
5. DQN-O - Similar to DQN but continues to learn during testing phase too.

![Results](./media/paper_stoa_res.png)
<br>
#### Related Work
1. Forward Prediction for Physical Reasoning - https://arxiv.org/abs/2006.10734
2. Causal World Models by Unsupervised Deconfounding of Physical Dynamics - https://arxiv.org/pdf/2012.14228.pdf

# Problem Summary

Given the initial scene find coordinate $(x_i, y_i)$ to place a ball of radius r in the environment such that by the end of the simulation we reach the goal state. Goal state is defined as the state where dynamic goal subject(2 or blue subject) touches static goal object (4) or dynamic goal object(3). Both 4 and 3 are green objects.

### Challenges in this problem
1. Assuming we have a perfect forward prediction model, it only helps in validating if a hypothesis $(x_i, y_i, r)$. It is doesn't help with coming up with $(x_i, y_i, r)$, so we are left with a large search space. Humans tackle this problem by reasoning with a qualitative physics model, so the challenge is not learning the physics of the environment but reasoning with this information

2. Sparse rewards - Only a handful of actions lead to a reward and they are concentrated around the objects of interest. All the three variables x, y, r are strongly correlated to each other. Many of the actions are invalid too. For example any action which touches borders of scene or any object in the scene are invalid.

![Actions](./media/task_act.png)

3. Uninformative or Vague rewards - Rewards we get back are binary and doesn't give information on how far it is from the goal state. So we need a way to encode the goal into a low-dimensional vector space which allows us to
    1. Provide feedback on how far agent is from the goal qualitatively.
    2. Provides a way to backtrack the actions to adjust the initial action

### Summary
So this problem can be cast as a search problem. I think we need the following
1. A framework to encode goal which gives more informative feedback at the end of simulation. 
2. An efficient way to backtrack from the intermediate frames provided.